In [16]:
import os
import re
import json
import openpyxl
import numpy as np
import pandas as pd
from datetime import datetime

In [17]:
os.makedirs("data/processed", exist_ok=True)
raw_folder = 'data/raw'

all_shoes = []
print(len(os.listdir(raw_folder)))
for filename in os.listdir(raw_folder):
    if filename.endswith(".json"):
        filepath = os.path.join(raw_folder, filename)
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                shoe_data = json.load(f)
                all_shoes.append(shoe_data)
        except Exception as e:
            print(f"Erro ao processar {filename}: {e}")

df = pd.DataFrame(all_shoes)

df.to_csv("data/processed/shoes_dataset.csv", index=False)

557


In [18]:
df.head()

,url,shoe_name,brand,total_score,pros,cons,RR_verdict,awards,external_ratings,review_date,...,Popularity,PopularityPercentile,Rocker,Lightweight,Plate,Trail terrain,Lug depth,Heel stack lab,For heavy runners,Waterproofing
0,https://runrepeat.com/adidas-4dfwd,Adidas 4DFWD,Adidas,90,Bouncier than most 4D shoes; Comfortable in-sh...,Upper stretches a lot over time; Lacks breatha...,The Adidas 4DFWD impressed us from the start w...,"Top 20% in road running shoes, Top 22% in Adid...","92 / 100 from 7,941 users | 86 / 100 from 16 e...","Aug 12, 2021 - updated Oct 11, 2023",...,#255,Top 46%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://runrepeat.com/adidas-4dfwd-2,Adidas 4DFWD 2,Adidas,90,Unique midsole construction; Supportive fit; C...,Not for the budget-minded; Very heavy; Not for...,Bridging the gap between running shoes and sne...,"Top 22% in road running shoes, Top 28% in Adid...","91 / 100 from 12,895 users | 86 / 100 from 9 e...","Oct 24, 2022 - updated Oct 11, 2023",...,#462,Bottom 17%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://runrepeat.com/adidas-4dfwd-3,Adidas 4DFWD 3,Adidas,88,Extremely comfortable; Excellent traction; Gre...,Quite heavy; Not good for high speeds or long ...,The third iteration of the Adidas 4DFWD contin...,Top pick in best Adidas running shoes (2024),"93 / 100 from 5,481 users | 89 / 100 from 1 ex...","Feb 26, 2024",...,#299,Bottom 7%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://runrepeat.com/adidas-adistar,Adidas Adistar,Adidas,88,Stable landings; Maxed-out cushion; Relaxed an...,Could be more responsive; Firm for heel strike...,Often dubbed as Adidas' version of the Hoka Bo...,,"91 / 100 from 7,934 users | 81 / 100 from 13 e...","Mar 18, 2022 - updated Oct 12, 2023",...,#392,Bottom 29%,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://runrepeat.com/adidas-adistar-20,Adidas Adistar 2.0,Adidas,85,Cushioned for long miles; Supportive for a neu...,Heavier than average; Firm heel landings,The Adidas Adistar 2.0 is a neutral daily trai...,Top pick in best running shoes for plantar fas...,"88 / 100 from 5,466 users | 88 / 100 from 1 ex...","Jun 20, 2023",...,#553,Bottom 1%,✓,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def snake_case(col):
    col = re.sub(r'(?<=[a-z])(?=[A-Z0-9])', '_', col)
    col = re.sub(r'(?<=[A-Z])(?=[A-Z][a-z])', '_', col)
    col = col.replace(' ', '_').replace('/', '_')
    col = re.sub(r'_+', '_', col) 
    return col.lower().strip()

In [20]:
df.columns = [snake_case(col) for col in df.columns]

In [21]:
df.columns 

Index(['url', 'shoe_name', 'brand', 'total_score', 'pros', 'cons',
       'rr_verdict', 'awards', 'external_ratings', 'review_date', 'terrain',
       'arch', 'use', 'shoe_weight', 'toe_drop', 'foot_height',
       'audience_score', 'price', 'pace', 'arch_support', 'weight_lab',
       'drop_lab', 'strike_pattern', 'size', 'midsole_softness',
       'difference_in_midsole_softness_in_cold', 'toebox_durability',
       'heel_padding_durability', 'outsole_durability', 'breathability',
       'width_fit', 'toebox_width', 'stiffness',
       'difference_in_stiffness_in_cold', 'torsional_rigidity',
       'heel_counter_stiffness', 'heel_lab', 'forefoot_lab',
       'widths_available', 'orthotic_friendly', 'season', 'removable_insole',
       'ranking', 'ranking_percentile', 'popularity', 'popularity_percentile',
       'rocker', 'lightweight', 'plate', 'trail_terrain', 'lug_depth',
       'heel_stack_lab', 'for_heavy_runners', 'waterproofing'],
      dtype='object')

In [22]:
def extrair_peso(texto):
    pesos = re.findall(r'(\d+\.?\d*)g', str(texto))
    return np.mean([float(p) for p in pesos]) if pesos else np.nan

In [23]:
def extrair_valor_masculino(texto, unidade):
    if not isinstance(texto, str):
        return np.nan
    match = re.search(r'Men:\s*[\d\.]+\s*' + unidade + r'\s*/\s*(\d+\.?\d*)', texto)
    if match:
        return float(match.group(1))
    match = re.search(r'(\d+\.?\d*)\s*' + unidade, texto)
    if match:
        return float(match.group(1))
    return np.nan

In [ ]:
if 'external_ratings' in df.columns:
    df['user_rating'] = df['external_ratings'].str.extract(r'(\d+)\s*/\s*100\s*from\s*[\d,]+\s*users')[0].astype(float)
    df['expert_rating'] = df['external_ratings'].str.extract(r'\|\s*(\d+)\s*/\s*100\s*from\s*[\d,]+\s*expert[s]?')[0].astype(float)

    df['user_review_count'] = df['external_ratings'].str.extract(r'(\d+)\s*/\s*100\s*from\s*([\d,]+)\s*users')[1].str.replace(',', '').astype(float)
    df['expert_review_count'] = df['external_ratings'].str.extract(r'\|\s*\d+\s*/\s*100\s*from\s*([\d,]+)\s*expert[s]?')[0].str.replace(',', '').astype(float)

if 'review_date' in df.columns:
    df['review_date'] = pd.to_datetime(df['review_date'].str.extract(r'(\w{3,9} \d{1,2}, \d{4})')[0], errors='coerce')

if 'use' in df.columns:
    df['use'] = df['use'].str.replace('|', '', regex=False).str.split(',')

if 'shoe_weight' in df.columns:
    df['shoe_weight_g'] = df['shoe_weight'].apply(lambda x: extrair_valor_masculino(x, 'g'))

if 'toe_drop' in df.columns:
    df['toe_drop_mm'] = df['toe_drop'].apply(lambda x: extrair_valor_masculino(x, 'mm'))

if 'foot_height' in df.columns:
    df['foot_height_mm'] = df['foot_height'].apply(lambda x: extrair_valor_masculino(x, 'mm'))

if 'audience_score' in df.columns:
    df['audience_score'] = df['audience_score'].str.extract(r'(\d+)').astype(float)

if 'price' in df.columns:
    df['price'] = df['price'].str.replace(r'[\$]', '', regex=True).astype(float)

if 'pace' in df.columns:
    df['pace'] = df['pace'].str.replace('|', '', regex=False).str.split(',')

if 'ranking' in df.columns:
    df['ranking'] = df['ranking'].str.replace('#', '', regex=False).astype(float)

if 'popularity' in df.columns:
    df['popularity'] = df['popularity'].str.replace('#', '', regex=False).astype(float)

if 'total_score' in df.columns:
    df['total_score'] = pd.to_numeric(df['total_score'], errors='coerce')

if 'drop_lab' in df.columns:
    df['drop_mm'] = df['drop_lab'].str.extractall(r'(\d+\.?\d*)').groupby(level=0)[0].apply(lambda x: x.astype(float).mean())

for campo in ['pros', 'cons', 'awards']:
    if campo in df.columns:
        df[campo] = df[campo].str.split(';|,')

df['num_pros'] = df['pros'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df['num_cons'] = df['cons'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df['num_awards'] = df['awards'].apply(lambda x: len(x) if isinstance(x, list) else 0)

if 'plate' in df.columns:
    df['has_carbon_plate'] = df['plate'].str.contains('carbon', case=False, na=False)


In [25]:
ordered = [
    # Identificação e data
    'shoe_name',
    'url',
    'brand',
    'review_date',

    # Preço
    'price',

    # Notas e avaliações (processadas primeiro)
    'total_score',
    'audience_score',
    'user_rating',
    'expert_rating',

    # Ranking/popularidade
    'ranking',
    'popularity',
    'ranking_percentile',
    'popularity_percentile',

    # Percepção qualitativa
    'pros',
    'cons',
    'awards',
    'rr_verdict',

    'num_pros',
    'num_cons',
    'num_awards',
    
    # Número de reviews
    'user_review_count',
    'expert_review_count',
    
    # Características gerais
    'terrain',
    'arch',
    'arch_support',
    'use',
    'pace',
    'season',

    # Medidas processadas primeiro
    'shoe_weight_g',
    'toe_drop_mm',
    'foot_height_mm',

    # Alturas e laboratórios
    'heel_stack_lab',
    'heel_lab',
    'forefoot_lab',

    # Materiais e construção
    'midsole_softness',
    'difference_in_midsole_softness_in_cold',
    'stiffness',
    'difference_in_stiffness_in_cold',
    'torsional_rigidity',
    'heel_counter_stiffness',
    'toebox_durability',
    'heel_padding_durability',
    'outsole_durability',
    'breathability',
    'lightweight',
    'plate',
    'rocker',
    'waterproofing',

    # Ajuste
    'size',
    'width_fit',
    'toebox_width',
    'widths_available',
    'orthotic_friendly',
    'for_heavy_runners',

    # Outros dados técnicos
    'drop_mm',
    'drop_lab',
    'weight_lab',
    'strike_pattern',
    'trail_terrain',
    'lug_depth',
    'has_carbon_plate',
    'removable_insole',
    
    # Medidas brutas depois
    'toe_drop',
    'foot_height',
    'shoe_weight',
]


In [26]:
df = df[ordered]

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 65 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   shoe_name                               557 non-null    object        
 1   url                                     557 non-null    object        
 2   brand                                   557 non-null    object        
 3   review_date                             557 non-null    datetime64[ns]
 4   price                                   557 non-null    float64       
 5   total_score                             556 non-null    float64       
 6   audience_score                          556 non-null    float64       
 7   user_rating                             557 non-null    float64       
 8   expert_rating                           505 non-null    float64       
 9   ranking                                 557 non-null  

In [ ]:
df.to_csv("data/processed/shoes_processed.csv", index=False)
df.to_excel("data/processed/shoes_processed.xlsx", index=False)


Arquivos 'shoes_processed.csv' e 'shoes_processed.xlsx' salvos com sucesso!


In [60]:
import pandas as pd
import nltk
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
columns = [    
    'shoe_name',
    'brand',
    'review_date',

    'total_score',

    'ranking',
    'popularity',

    'pros',
    'cons',
    'rr_verdict',
    
    'user_review_count',
]

ds = df[columns].copy()

### Pros, Cons and RR veredict anaylsis

In [ ]:
def list_to_string_with_dot(cell):
    if isinstance(cell, list):
        return '. '.join(item.strip() for item in cell if isinstance(item, str))
    return str(cell) 

ds["pros"] = ds["pros"].apply(list_to_string_with_dot)
ds["cons"] = ds["cons"].apply(list_to_string_with_dot)
ds["rr_verdict"] = ds["rr_verdict"].apply(list_to_string_with_dot)


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm import tqdm 

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    if not isinstance(text, str):
        return {"compound": 0, "pos": 0, "neu": 0, "neg": 0}
    return sia.polarity_scores(text)

tqdm.pandas()
ds["sentiment_pros"] = ds["pros"].progress_apply(lambda x: get_sentiment_scores(x)["compound"])
ds["sentiment_cons"] = ds["cons"].progress_apply(lambda x: get_sentiment_scores(x)["compound"])
ds["sentiment_veredict"] = ds["rr_verdict"].progress_apply(lambda x: get_sentiment_scores(x)["compound"])

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
100%|██████████| 557/557 [00:00<00:00, 659.59it/s]


In [ ]:
media_sentimento = ds.groupby('brand')['sentiment_veredict'].mean().reset_index().sort_values(by='sentiment_veredict', ascending=False)
media_sentimento.reset_index(drop=True, inplace=True)
media_sentimento

,brand,sentiment_veredict
0,Scarpa,0.980200
1,Allbirds,0.959433
2,NNormal,0.946800
3,Inov8,0.940600
4,Skechers,0.932580
5,Reebok,0.921820
6,Topo Athletic,0.914250
7,Xero Shoes,0.899450
8,ASICS,0.875894
9,Salomon,0.875247
